# 1 read data from text file

In [3]:
#======================== import module order as three level========================
import os
from os.path import join, exists, split
import numpy as np
import data_helpers
from keras.callbacks import EarlyStopping
from keras.models import model_from_json
from keras.callbacks import TensorBoard
import tensorflow as tf

Using TensorFlow backend.


In [8]:
#TODO: After complete all training, use argparse to store the params.
positive_data_file = "../data/rt-polaritydata/rt-polarity.pos"
negtive_data_file = "../data/rt-polaritydata/rt-polarity.neg"

# Load data
print("Loading data...")
x_text, y = data_helpers.load_data_and_labels(positive_data_file, negtive_data_file)


Loading data...


In [12]:
length = [len(sent) for sent in x_text]

In [14]:
# max length
max(length)

56

In [17]:
# min length
min(length)

1

In [18]:
# average
sum(length) / len(length)

20.38585631213656

Because the length is not too loog, so we set the sequence length as 56. 

# 2 string to index

In [20]:
#=======================Convert string to index================
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenizer
tk = Tokenizer(num_words=None, oov_token='UNK')
tk.fit_on_texts(x_text)

In [22]:
tk.word_index['UNK']

18765

In [30]:
# Convert string to index 
sequences = tk.texts_to_sequences(x_text)

In [31]:
print(train_sequences[:2])

[[1, 565, 7, 2633, 6, 22, 1, 3369, 887, 8, 100, 5598, 4, 11, 65, 8, 240, 6, 73, 3, 3913, 57, 2948, 34, 1489, 2393, 2, 2394, 10111, 1708, 7197, 42, 937, 10112], [1, 3370, 2181, 7198, 5, 1, 3914, 5, 1, 2949, 4631, 7, 39, 1075, 11, 3, 7199, 5, 823, 888, 4632, 2634, 685, 246, 66, 1076, 889, 8, 4633, 686, 5, 1241, 1709, 1709, 5599, 8, 480, 1490]]


In [32]:
# Padding
sequences_pad = pad_sequences(sequences, maxlen=56, padding='post')

In [33]:
print(sequences_pad[:2])

[[    1   565     7  2633     6    22     1  3369   887     8   100  5598
      4    11    65     8   240     6    73     3  3913    57  2948    34
   1489  2393     2  2394 10111  1708  7197    42   937 10112     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [    1  3370  2181  7198     5     1  3914     5     1  2949  4631     7
     39  1075    11     3  7199     5   823   888  4632  2634   685   246
     66  1076   889     8  4633   686     5  1241  1709  1709  5599     8
    480  1490     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]]


In [34]:
sequences_pad.shape

(10662, 56)

In [35]:
len(tk.word_index)

18765

# 3 word embedding 

The vocab size is 18765, so we will use Glove to build a 18766x50 word embedding weights. First row is represent padding which contain all 0

In [37]:
glove_path = 'glove.6B'
with open(os.path.join(glove_path, 'glove.6B.50d.txt')) as f:
    i = 0
    for line in f.readlines():
        print(line)
#         print(line.split())
        i += 1
        if i > 1:
            break


the 0.418 0.24968 -0.41242 0.1217 0.34527 -0.044457 -0.49688 -0.17862 -0.00066023 -0.6566 0.27843 -0.14767 -0.55677 0.14658 -0.0095095 0.011658 0.10204 -0.12792 -0.8443 -0.12181 -0.016801 -0.33279 -0.1552 -0.23131 -0.19181 -1.8823 -0.76746 0.099051 -0.42125 -0.19526 4.0071 -0.18594 -0.52287 -0.31681 0.00059213 0.0074449 0.17778 -0.15897 0.012041 -0.054223 -0.29871 -0.15749 -0.34758 -0.045637 -0.44251 0.18785 0.0027849 -0.18411 -0.11514 -0.78581

, 0.013441 0.23682 -0.16899 0.40951 0.63812 0.47709 -0.42852 -0.55641 -0.364 -0.23938 0.13001 -0.063734 -0.39575 -0.48162 0.23291 0.090201 -0.13324 0.078639 -0.41634 -0.15428 0.10068 0.48891 0.31226 -0.1252 -0.037512 -1.5179 0.12612 -0.02442 -0.042961 -0.28351 3.5416 -0.11956 -0.014533 -0.1499 0.21864 -0.33412 -0.13872 0.31806 0.70358 0.44858 -0.080262 0.63003 0.32111 -0.46765 0.22786 0.36034 -0.37818 -0.56657 0.044691 0.30392



In [38]:
# read glove to embedding
embeddings_index = {}
f = open(os.path.join(glove_path, 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [40]:
embeddings_index['the']

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)

In [44]:
embeddings_index['the']

-5.2395954

In [43]:
mu, sigma = 0, 0.1 # mean and standard deviation
s = np.random.normal(mu, sigma, 50)
s

array([-0.16135411,  0.03557489, -0.09522059, -0.1242005 ,  0.06621048,
       -0.09528465, -0.12728056,  0.07979754,  0.09633083,  0.20050312,
       -0.05113975, -0.00972726, -0.02855471, -0.01472568, -0.03357385,
        0.1873512 ,  0.03185613, -0.04298057, -0.17279828, -0.20144806,
       -0.03849123,  0.08185671,  0.16343176,  0.09456343, -0.04212118,
       -0.03696294, -0.11233464,  0.03048478,  0.02966199, -0.06009029,
       -0.07345058, -0.03272793, -0.04158637, -0.06199308, -0.15955818,
        0.16222283, -0.16236277, -0.09712829,  0.01742469,  0.00733383,
       -0.00704928,  0.0138043 , -0.06280551, -0.13909955, -0.06535425,
        0.05937353,  0.23325249,  0.06374299,  0.09044405, -0.01901892])

In [46]:
# Add random vector to 'UNK'
embeddings_index['UNK'] = np.random.normal(mu, sigma, 50)

In [48]:
embedding_dim = 50

# use embeddings_index to build our embedding_matrix
embedding_matrix = np.zeros((len(tk.word_index)+1, embedding_dim)) # fist row represent 0, pad
for word, i in tk.word_index.items():
    embedding_vector = embeddings_index.get(word) # if not find in the dict, return None
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_vector = embeddings_index.get('UNK')
        embedding_matrix[i] = embedding_vector

In [49]:
embedding_matrix.shape

(18766, 50)